# Aula 4 - Jogos e satisfação de restrições

## Parte 1 - Jogos

Vamos implementar um jogo e o algoritmo Alpha-Beta Pruning. Para começar, abaixo seguem algumas definições iniciais.

In [0]:
import numpy as np
import random
from collections import namedtuple
from google.colab import output
import time

Inicialmente, criaremos uma classe genérica para jogos com `n` jogadores com movimentos alternados. Note que esta classe é parecida com a definição de problemas que vimos antes. A diferença é que agora precisamos definir uma utilidade para cada estado e também um teste de estado final; anteriormente, tínhamos o custo de um caminho e um teste de objetivo. Para criar um novo jogo, basta criar uma subclasse de `Game` e implementar os métodos `actions`, `result`, `utility` e `terminal_test`. As funções `display` e `to_move` podem ser sobrescritas, se necessário, por suas correspondentes específicas ao problema. Ao criar a subclasse, é necessário ainda definir o estado inicial `self.initial` no construtor da classe.

In [0]:
class Game:
    
    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)
        
    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players, animate=True):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if animate:
                  self.display(state)
                if self.terminal_test(state):
                    if not animate:
                      self.display(state)
                    return self.utility(state, self.to_move(self.initial))

Agora, vamos criar uma classe para o jogo da velha (Tic-Tac-Toe). O construtor recebe três argumentos. Os argumentos `h` e `v` denotam a largura e altura do tabuleiro. O argumento `k` denota o número de peças que devem ser alinhadas para se ganhar o jogo. Note, portanto, que esta é uma implementação genérica, permitindo não apenas tabuleiros de 3x3. Em nossa implementação, o primeiro jogador utiliza X e o segundo utiliza O.

In [0]:
# state representation as tuple, where 'to_move' denotes who moves next, 'utility' 
# denotes the utility of the current board state, 'board' denotes the state utility, 
# and 'moves' denotes which movements are possible from current state
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

# game class
class TicTacToe(Game):
    """Play TicTacToe on an h x v board, with Max (first player) playing 'X'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'."""

    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """Legal moves are any square not yet taken."""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        string = ''
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                string += ' ' + board.get((x, y), '.') + ' '
                if y != self.v:
                  string += '|'
            string += '\n'
            if x != self.h:
              string += '-'*(self.v*4-1) + '\n'
        output.clear()
        print(string, flush=True)
        time.sleep(1)

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

Para criar uma instância do jogo, basta fazer como segue.

In [4]:
ttt = TicTacToe()

# print the initial state board
ttt.display(ttt.initial)

 . | . | . 
-----------
 . | . | . 
-----------
 . | . | . 



Agora podemos criar os jogadores. Nosso primeiro jogador é bem simples, jogando de forma totalmente aleatória independente do estado atual. Naturalmente, apenas ações possíveis são escolhidas. 

In [0]:
def random_player(game, state):
    return random.choice(game.actions(state)) if game.actions(state) else None

Nosso segundo jogador utiliza o algoritmo Alpha-Beta Pruning. Para tal, o jogador recebe um estado e explora toda a árvore do jogo a partir dali (com exceção daquelas porções que são podadas). 

In [0]:
def alpha_beta_player(game, state):
    return alpha_beta_search(state, game)

def alpha_beta_search(state, game):
    
    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

Finalmente, podemos iniciar o jogo. Para tal, basta executar a função `play_game` da nossa instância de `TicTacToe`. A função retorna a utilidade final sob a perspectiva do jogador 1. 

Neste primeiro teste, vamos colocar dois jogadores aleatórios para competir.

In [7]:
result = ttt.play_game(random_player, random_player)
winner = {1: 'Player 1', 0: 'Draw', -1: 'Player 2'}
print('Result: %d' % result)
print('Winner: %s' % winner[result])

 O | X | X 
-----------
 X | X | O 
-----------
 X | O | O 

Result: 1
Winner: Player 1


Note que, como os jogadores são aleatórios, o resultado do jogo varia de uma execução para outra. Para testar, rode o código acima mais de uma vez e compare os resultados.

Agora, vamos ver como o jogador aleatório se sai contra o jogador `alpha_beta_player`.

In [8]:
result = ttt.play_game(alpha_beta_player, random_player)
winner = {1: 'Player 1', 0: 'Draw', -1: 'Player 2'}
print('Result: %d' % result)
print('Winner: %s' % winner[result])

 X | O | . 
-----------
 X | X | O 
-----------
 O | . | X 

Result: 1
Winner: Player 1


Note que o jogador aleatório tem poucas chances contra o seu oponente. Na maioria das vezes, é o `alpha_beta_player` quem ganha. Para testarmos melhor, vamos repetir o jogo 20 vezes. Ao final, mostraremos o somatório dos resultados dos jogos. Se o somatório for menor que 20, então significa que o jogador aleatório ganhou pelo menos uma vez.

In [9]:
repetitions = 20
summation = 0
for i in range(repetitions):
  summation += ttt.play_game(alpha_beta_player, random_player, animate=False)

print('Result: %d/%d' % (summation, repetitions))

 X | O | . 
-----------
 X | X | X 
-----------
 . | O | O 

Result: 20/20


Finalmente, vamos ver agora o que acontece quando dois jogadores do tipo `alpha_beta_player` jogam um contra o outro.

In [10]:
result = ttt.play_game(alpha_beta_player, alpha_beta_player)
winner = {1: 'Player 1', 0: 'Draw', -1: 'Player 2'}
print('Result: %d' % result)
print('Winner: %s' % winner[result])

 X | X | O 
-----------
 O | O | X 
-----------
 X | O | X 

Result: 0
Winner: Draw


Neste cenário, note que o jogo sempre terminará em empate. Como ambos jogadores conhecem as melhores jogadas a cada estado, o empate se torna inevitável.

### Exercício 1

Calcule os valores Minimax de cada estado da árvore de jogo abaixo. 

![alt text](https://i.imgur.com/1mrTV5y.png)

Os valores Minimax dos estados são:
* Estado 'A': 4
* Estado 'B': -5
* Estado 'C': 4
* Estado 'D': -1
* Estado 'E': 0

### Exercício 2

Com base no algoritmo Alpha-Beta pruning, encontre os valores alpha e beta de cada estado da árvore de jogo abaixo. Lembre-se que este algoritmo poda porções da árvore; portanto, não deixe de indicar as porções podadas pelo algoritmo.

![alt text](https://i.imgur.com/zII14oY.png)

Os valores alpha-beta dos estados são:
* Estado 'A': [10, 10]
* Estado 'B': [10, 10]
* Estado 'C': [-inf, 8]
* Estado 'D': [25, 25]
* Estado 'E': [10, 10]
* Estado 'F': [8, 8]
* Estado 'G': [-inf, +inf]
* Estado 'H': [25, 25]
* Estado 'I': [-inf, -31]
* Estado 'J': [10, 10]
* Estado 'K': [-inf, -5]
* Estado 'L': [8, 8]
* Estado 'M': [-inf, 0]
* Estado 'N': [-inf, +inf]
* Estado 'O': [-inf, +inf]

As porções podadas da árvore são (identifiquei as arestas com base no nome das ações; cada aresta listada abaixo indica que a subárvore correspondente foi eliminada):
* Aresta 'i2'
* Aresta 'k2'
* Aresta 'm2'
* Aresta 'c2'


### Exercício 3

Crie uma classe para representar o jogo do Exercício 1. Em seguida, teste os algoritmos `alpha_beta_player` e `random_player` e comente os resultados.

In [0]:
GameState = namedtuple('GameStateEx1', 'to_move, utility, node, moves')

class JogoEx1(Game):
    
    def __init__(self):
        self.moves = {'A': {'a1': 'B',   'a2': 'C',   'a3': 'D',   'a4': 'E'},
                      'B': {'b1': 'FB1', 'b2': 'FB2', 'b3': 'FB3', 'b4': 'FB4'},
                      'C': {'c1': 'FC1', 'c2': 'FC2', 'c3': 'FC3', 'c4': 'FC4'},
                      'D': {'d1': 'FD1', 'd2': 'FD2', 'd3': 'FD3', 'd4': 'FD4'},
                      'E': {'e1': 'FE1', 'e2': 'FE2', 'e3': 'FE3', 'e4': 'FE4'}}
                      
        self.utilities = {'FB1': -5, 'FB2': 8, 'FB3': 0,  'FB4': 3,
                          'FC1': 10, 'FC2': 8, 'FC3': 6,  'FC4': 4,
                          'FD1': 0,  'FD2': 2, 'FD3': -1, 'FD4': 5,
                          'FE1': 3,  'FE2': 2, 'FE3': 20, 'FE4': 0}
        
        self.initial = GameState(to_move='J1', utility=0, node='A', moves=list(self.moves['A'].keys()))

    # igual
    def actions(self, state):
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        
        new_node = self.moves[state.node][move]
        return GameState(to_move=('J2' if state.to_move == 'J1' else 'J1'),
                         utility=(self.utilities[new_node] if new_node in self.utilities else 0),
                         node=new_node, 
                         moves=(list(self.moves[new_node].keys()) if new_node in self.moves else None))

    def utility(self, state, player):
        return state.utility if player == 'J1' else -state.utility

    def terminal_test(self, state):
        return not state.node in self.moves

In [12]:
jex1 = JogoEx1()
result = jex1.play_game(alpha_beta_player, alpha_beta_player)

print('Player 1: %d' % (result))
print('Player 2: %d' % (-result))

if result > 0:
  print('Player 1 wins!')
elif result < 0:
  print('Player 2 wins!')
else:
  print('Draw!')

GameStateEx1(to_move='J2', utility=0, node='C', moves=['c1', 'c2', 'c3', 'c4'])
GameStateEx1(to_move='J1', utility=4, node='FC4', moves=None)
Player 1: 4
Player 2: -4
Player 1 wins!


### Exercício extra

Generalize a classe de jogo criada no Exercício 3 para que ela acomode jogos de múltiplos rounds, como o do Exercício 2. Em seguida, teste os algoritmos `alpha_beta_player` e `random_player` e comente os resultados.


In [0]:
# your answer here

## Parte 2 - Satisfação de restrições

Primeiramente, vamos criar algumas funções e classes de apoio.

In [0]:
import random
import numpy as np
from collections import defaultdict

def first(iterable, default=None):
    """Return the first element of an iterable; or default."""
    return next(iter(iterable), default)

def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(map(bool, seq))

class UniversalDict:
    """A universal dict maps any key to the same value. We use it here
    as the domains dict for CSPs in which all variables have the same domain.
    >>> d = UniversalDict(42)
    >>> d['life']
    42
    """

    def __init__(self, value): 
      self.value = value

    def __getitem__(self, key): 
      return self.value

    def __repr__(self): 
      return '{{Any: {0!r}}}'.format(self.value)

class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

Agora, vamos criar uma classe que formalize um problema de satisfação de restrições (CSP).

In [0]:
class CSP(Problem):
    """This class describes finite-domain Constraint Satisfaction Problems.
    A CSP is specified by the following inputs:
        variables   A list of variables; each is atomic (e.g. int or string).
        domains     A dict of {var:[possible_value, ...]} entries.
        neighbors   A dict of {var:[var,...]} that for each variable lists
                    the other variables that participate in constraints.
        constraints A function f(A, a, B, b) that returns true if neighbors
                    A, B satisfy the constraint when they have values A=a, B=b

    In the textbook and in most mathematical definitions, the
    constraints are specified as explicit pairs of allowable values,
    but the formulation here is easier to express and more compact for
    most cases. (For example, the n-Queens problem can be represented
    in O(n) space using this notation, instead of O(N^4) for the
    explicit representation.) In terms of describing the CSP as a
    problem, that's all there is.

    However, the class also supports data structures and methods that help you
    solve CSPs by calling a search function on the CSP. Methods and slots are
    as follows, where the argument 'a' represents an assignment, which is a
    dict of {var:val} entries:
        assign(var, val, a)     Assign a[var] = val; do other bookkeeping
        unassign(var, a)        Do del a[var], plus other bookkeeping
        nconflicts(var, val, a) Return the number of other variables that
                                conflict with var=val
        curr_domains[var]       Slot: remaining consistent values for var
                                Used by constraint propagation routines.
    The following methods are used only by graph_search and tree_search:
        actions(state)          Return a list of actions
        result(state, action)   Return a successor of state
        goal_test(state)        Return true if all constraints satisfied
    The following are just for debugging purposes:
        nassigns                Slot: tracks the number of assignments made
        display(a)              Print a human-readable representation
    """

    def __init__(self, variables, domains, neighbors, constraints):
        """Construct a CSP problem. If variables is empty, it becomes domains.keys()."""
        variables = variables or list(domains.keys())
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraints = constraints
        self.initial = ()
        self.curr_domains = None
        self.nassigns = 0

    def assign(self, var, val, assignment):
        """Add {var: val} to assignment; Discard the old value if any."""
        assignment[var] = val
        self.nassigns += 1

    def unassign(self, var, assignment):
        """Remove {var: val} from assignment.
        DO NOT call this if you are changing a variable to a new value;
        just call assign for that."""
        if var in assignment:
            del assignment[var]

    def nconflicts(self, var, val, assignment):
        """Return the number of conflicts var=val has with other variables."""

        # Subclasses may implement this more efficiently
        def conflict(var2):
            return (var2 in assignment and
                    not self.constraints(var, val, var2, assignment[var2]))

        return count(conflict(v) for v in self.neighbors[var])

    def display(self, assignment):
        """Show a human-readable representation of the CSP."""
        # Subclasses can print in a prettier way, or display with a GUI
        print('CSP:', self, 'with assignment:', assignment)

    # These methods are for the tree and graph-search interface:

    def actions(self, state):
        """Return a list of applicable actions: nonconflicting
        assignments to an unassigned variable."""
        if len(state) == len(self.variables):
            return []
        else:
            assignment = dict(state)
            var = first([v for v in self.variables if v not in assignment])
            return [(var, val) for val in self.domains[var]
                    if self.nconflicts(var, val, assignment) == 0]

    def result(self, state, action):
        """Perform an action and return the new state."""
        (var, val) = action
        return state + ((var, val),)

    def goal_test(self, state):
        """The goal is to assign all variables, with all constraints satisfied."""
        assignment = dict(state)
        return (len(assignment) == len(self.variables)
                and all(self.nconflicts(variables, assignment[variables], assignment) == 0
                        for variables in self.variables))

    # These are for constraint propagation

    def support_pruning(self):
        """Make sure we can prune values from domains. (We want to pay
        for this only if we use it.)"""
        if self.curr_domains is None:
            self.curr_domains = {v: list(self.domains[v]) for v in self.variables}

    def suppose(self, var, value):
        """Start accumulating inferences from assuming var=value."""
        self.support_pruning()
        removals = [(var, a) for a in self.curr_domains[var] if a != value]
        self.curr_domains[var] = [value]
        return removals

    def prune(self, var, value, removals):
        """Rule out var=value."""
        self.curr_domains[var].remove(value)
        if removals is not None:
            removals.append((var, value))

    def choices(self, var):
        """Return all values for var that aren't currently ruled out."""
        return (self.curr_domains or self.domains)[var]

    def infer_assignment(self):
        """Return the partial assignment implied by the current inferences."""
        self.support_pruning()
        return {v: self.curr_domains[v][0]
                for v in self.variables if 1 == len(self.curr_domains[v])}

    def restore(self, removals):
        """Undo a supposition and all inferences from it."""
        for B, b in removals:
            self.curr_domains[B].append(b)

Agora podemos criar o algoritmo Backtracking-Search. Este algoritmo é implementado pela função `backtracking_search`, que recebe um CSP (`csp`), além de funções para seleção de variáveis (`select_unassigned_variable`), seleção de domínios de variáveis (`order_domain_values`) e inferência (`inference`). Por padrão, nossa implementação recebe as funções `first_unassigned_variable` (que retorna a primeira variável livre), `unordered_domain_values` (que retorna todos os valores disponíveis) e `no_inference` (que não faz qualquer inferência). Ao final, nossa função `backtracking_search` retorna a alocação encontrada (ou `None`, caso nenhuma alocação válida tenha sido encontrada).


In [0]:
def first_unassigned_variable(assignment, csp):
    return first([var for var in csp.variables if var not in assignment])

def unordered_domain_values(var, assignment, csp):
    return csp.choices(var)

def no_inference(csp, var, value, assignment, removals):
    return True

def backtracking_search(csp, select_unassigned_variable=first_unassigned_variable,
                        order_domain_values=unordered_domain_values, inference=no_inference):

    def backtrack(assignment):
        if len(assignment) == len(csp.variables):
            return assignment
        var = select_unassigned_variable(assignment, csp)
        for value in order_domain_values(var, assignment, csp):
            if 0 == csp.nconflicts(var, value, assignment):
                csp.assign(var, value, assignment)
                removals = csp.suppose(var, value)
                if inference(csp, var, value, assignment, removals):
                    result = backtrack(assignment)
                    if result is not None:
                        return result
                csp.restore(removals)
        csp.unassign(var, assignment)
        return None

    result = backtrack({})
    assert result is None or csp.goal_test(result)
    return result

Note que o algoritmo acima não é muito eficiente na seleção de variáveis/valores e no uso de inferência. Vamos resolver isso com técnica mais sofisticadas. 

Primeiramente, abaixo empregamos a heurística "minimum remaining values" (`mrv`), que recebe um CSP (`csp`) e retorna a variável com menos possibilidades de valores disponíveis.

In [0]:
def mrv(assignment, csp):
    return argmin_random_tie([v for v in csp.variables if v not in assignment],
                             key=lambda var: num_legal_values(csp, var, assignment))

def num_legal_values(csp, var, assignment):
    if csp.curr_domains:
        return len(csp.curr_domains[var])
    else:
        return count(csp.nconflicts(var, val, assignment) == 0 for val in csp.domains[var])

def argmin_random_tie(seq, key=lambda x: x):
    """Return a minimum element of seq; break ties at random."""
    items = list(seq)
    random.shuffle(items)
    return min(items, key=key)

Em termos de seleção de valores, abaixo apresentamos a heurística "least constrained value" (`lcv`), que recebe um CSP (`csp`), uma variável (`var`) e uma alocação parcial (`assignment`) e retorna o valor de `var` que restringe menos as demais variáveis.

In [0]:
def lcv(var, assignment, csp):
    return sorted(csp.choices(var), key=lambda val: csp.nconflicts(var, val, assignment))

Finalmente, abaixo apresentamos o mecanismo `forward_checking` para inferências. Este mecanismo recebe um CSP (`csp`), uma variável (`var`), um valor (`value`), uma alocação parcial (`assignment`) e uma lista de remoções (`removals`). O algoritmo então antecipa possíveis falhas e poda os valores correspondentes. 

In [0]:
def forward_checking(csp, var, value, assignment, removals):
    csp.support_pruning()
    for B in csp.neighbors[var]:
        if B not in assignment:
            for b in csp.curr_domains[B][:]:
                if not csp.constraints(var, value, B, b):
                    csp.prune(B, b, removals)
            if not csp.curr_domains[B]:
                return False
    return True

Definidos o algoritmo de backtracking e a formulação geral de CSPs, agora podemos criar uma classe para o problema N-Queens. Faremos isto através da classe `NQueensCSP`, subclasse de `CSP`, que consegue representar instâncias arbitrárias do problema (como a instância clássica 8-rainhas).

In [0]:
class NQueensCSP(CSP):
    """Make a CSP for the nQueens problem for search with min_conflicts.
    Suitable for large n, it uses only data structures of size O(n).
    Think of placing queens one per column, from left to right.
    That means position (x, y) represents (var, val) in the CSP.
    The main structures are three arrays to count queens that could conflict:
        rows[i]      Number of queens in the ith row (i.e val == i)
        downs[i]     Number of queens in the \ diagonal
                     such that their (x, y) coordinates sum to i
        ups[i]       Number of queens in the / diagonal
                     such that their (x, y) coordinates have x-y+n-1 = i
    We increment/decrement these counts each time a queen is placed/moved from
    a row/diagonal. So moving is O(1), as is nconflicts.  But choosing
    a variable, and a best value for the variable, are each O(n).
    If you want, you can keep track of conflicted variables, then variable
    selection will also be O(1).
    >>> len(backtracking_search(NQueensCSP(8)))
    8
    """

    def __init__(self, n):
        """Initialize data structures for n Queens."""
        CSP.__init__(self, list(range(n)), UniversalDict(list(range(n))),
                     UniversalDict(list(range(n))), self.queen_constraint)

        self.rows = [0] * n
        self.ups = [0] * (2 * n - 1)
        self.downs = [0] * (2 * n - 1)

    def nconflicts(self, var, val, assignment):
        """The number of conflicts, as recorded with each assignment.
        Count conflicts in row and in up, down diagonals. If there
        is a queen there, it can't conflict with itself, so subtract 3."""
        n = len(self.variables)
        c = self.rows[val] + self.downs[var + val] + self.ups[var - val + n - 1]
        if assignment.get(var, None) == val:
            c -= 3
        return c

    def assign(self, var, val, assignment):
        """Assign var, and keep track of conflicts."""
        oldval = assignment.get(var, None)
        if val != oldval:
            if oldval is not None:  # Remove old val if there was one
                self.record_conflict(assignment, var, oldval, -1)
            self.record_conflict(assignment, var, val, +1)
            CSP.assign(self, var, val, assignment)

    def unassign(self, var, assignment):
        """Remove var from assignment (if it is there) and track conflicts."""
        if var in assignment:
            self.record_conflict(assignment, var, assignment[var], -1)
        CSP.unassign(self, var, assignment)

    def record_conflict(self, assignment, var, val, delta):
        """Record conflicts caused by addition or deletion of a Queen."""
        n = len(self.variables)
        self.rows[val] += delta
        self.downs[var + val] += delta
        self.ups[var - val + n - 1] += delta

    def display(self, assignment):
        """Print the queens and the nconflicts values (for debugging)."""
        n = len(self.variables)
        for val in range(n):
            for var in range(n):
                if assignment.get(var, '') == val:
                    ch = 'Q'
                elif (var + val) % 2 == 0:
                    ch = '.'
                else:
                    ch = '-'
                print(ch, end=' ')
            print('    ', end=' ')
            for var in range(n):
                if assignment.get(var, '') == val:
                    ch = '*'
                else:
                    ch = ' '
                print(str(self.nconflicts(var, val, assignment)) + ch, end=' ')
            print()

    def queen_constraint(self, A, a, B, b):
      """Constraint is satisfied (true) if A, B are really the same variable,
      or if they are not in the same row, down diagonal, or up diagonal."""
      return A == B or (a != b and A + a != B + b and A - a != B - b)

De modo a facilitar a visualização das soluções, abaixo criamos ainda uma função de apoio. A função `plot_NQueens` recebe uma solução (`solution`) e a imprime de modo organizado.

In [0]:
def plot_NQueens(solution):
    n = len(solution)
    line = '-'*(n*4+1)
    print(line)
    for c in range(n):
      string = '|'
      for r in range(n):
        p = ' '
        if c == solution[r]:
          p = 'X'
        string += ' ' + p + ' |'
      print(string)
      print('-'*(n*4+1))

Agora podemos finalmente testar nosso algoritmo de backtracking no problema N-Queens. Abaixo, criamos a instância 8-rainhas. Em seguida, rodamos o algoritmo `backtracking_search` nesta instância, utilizando as 
heurísticas de seleção de variáveis/valores e de inferência criadas por último.

In [21]:
eight_queens = NQueensCSP(8)
solution = backtracking_search(
    eight_queens, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    inference=no_inference
    )
plot_NQueens(solution)
print(solution)

---------------------------------
|   |   |   |   | X |   |   |   |
---------------------------------
|   |   | X |   |   |   |   |   |
---------------------------------
|   |   |   |   |   |   |   | X |
---------------------------------
|   |   |   | X |   |   |   |   |
---------------------------------
|   |   |   |   |   |   | X |   |
---------------------------------
| X |   |   |   |   |   |   |   |
---------------------------------
|   |   |   |   |   | X |   |   |
---------------------------------
|   | X |   |   |   |   |   |   |
---------------------------------
{4: 0, 2: 1, 7: 2, 5: 6, 0: 5, 3: 3, 6: 4, 1: 7}


Como pode ser observado, o resultado encontrado é uma solução válida para o problema. 

Lembre-se que criamos funções simples e mais sofisticadas para a seleção de variáveis/valores e de inferência. Para poder comparar ambas, vamos testar uma instância maior do problema, o 20-rainhas. 

In [22]:
import time

twenty_queens1 = NQueensCSP(20)
twenty_queens2 = NQueensCSP(20)

start = time.time()
solution1 = backtracking_search(
    twenty_queens1, 
    select_unassigned_variable=first_unassigned_variable,
    order_domain_values=unordered_domain_values, 
    inference=no_inference
    )
runtime1 = time.time() - start
plot_NQueens(solution1)

start = time.time()
solution2 = backtracking_search(
    twenty_queens2, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    #inference=no_inference
    inference=forward_checking
    )
runtime2 = time.time() - start
plot_NQueens(solution2)

print('Runtime 1: %.2fs' % (runtime1))
print('Runtime 2: %.2fs' % (runtime2))

---------------------------------------------------------------------------------
| X |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
---------------------------------------------------------------------------------
|   |   |   | X |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
---------------------------------------------------------------------------------
|   | X |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
---------------------------------------------------------------------------------
|   |   |   |   | X |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
---------------------------------------------------------------------------------
|   |   | X |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
---------------------------------------------------------------------------------
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   | X |   |
----------------

Conforme pode ser observado acima, as heurísticas e inferências realizadas pelo segundo algoritmo o tornam consideravelmente mais rápido que a versão mais trivial do mesmo.

### Exercício 4

Modele o problema Sudoku como um CSP. Para isto, crie a classe `SudokuCSP`. Ao ser instanciada, esta classe deve receber como argumento uma instância do problema como o exemplo (a) da figura abaixo (vide Exercício 5 para representação desta instância). Ainda na inicialização, é preciso converter a instância recebida em um grafo seguindo a formalização de um CSP (vide classe correspondente). Dica: para denotar posições fixas (ex: no exemplo, a posição A3 tem valor 3 fixo), crie domínios com um valor único para estas variáveis.

![alt text](https://i.imgur.com/YRbZf7l.png)

In [0]:
class SudokuCSP(CSP):
    
    def __init__(self, instance):

        self.rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
        self.columns = list(range(1,10))

        # create the main elements of the CSP
        variables = []
        domains = {}
        neighbors = {}

        # populate the main elements of the CSP
        for row in self.rows:
            for col in self.columns:
                var = '%s%d' % (row, col)
                variables.append(var)
                domains[var] = list(range(1,10))
                neighbors[var] = self.list_neighbors_var(row, col)

        # process the variables already assigned in the instance
        for rowid, row in enumerate(self.rows):
            for colid, col in enumerate(self.columns):
                if instance[rowid][colid] != 0:
                    var = '%s%d' % (row, col)
                    domains[var] = [instance[rowid][colid]]
        
        # initialise the CSP
        CSP.__init__(self, 
                     variables, #list(range(n)), 
                     domains, #UniversalDict(list(range(n))),
                     neighbors, #UniversalDict(list(range(n))), 
                     self.sudoku_constraints)

    def list_neighbors_var(self, row, col):

        neighbors = []

        # same column
        for r in self.rows:
            neigh = '%s%d' % (r, col)
            if neigh not in neighbors:
                neighbors.append(neigh)

        # same row
        for c in self.columns:
            neigh = '%s%d' % (row, c)
            if neigh not in neighbors:
                neighbors.append(neigh)

        # same sub-region
        rowid = self.rows.index(row)
        reg_cs = list(range(int((col-1) / 3) * 3 + 1, int((col-1) / 3) * 3 + 3 + 1)) # columns in sub-region
        reg_rs = self.rows[int(rowid / 3) * 3 : int(rowid / 3) * 3 + 3] # rows in sub-region
        for r in reg_rs:
            for c in reg_cs:
                neigh = '%s%d' % (r, c)
                if neigh not in neighbors:
                    neighbors.append(neigh)

        return neighbors

    def sudoku_constraints(self, A, a, B, b):
        return A == B or (A != B and a != b)

    def display(self, assignment):
        rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
        columns = list(range(1,10))
        print('-'*37)
        for r in rows:
            line = '|'
            for c in columns:
                neigh = '%s%d' % (r, c)
                line += ' %d |' % (assignment[neigh])
            print(line)
            print('-'*37)

### Exercício 5

Resolva a instância de Sudoku apresentada na figura (a) do exercício 4 utilizando o algoritmo `backtracking_search`. Compare a execução do algoritmo com e sem as heurísticas de seleção de variáveis/valores e de inferência.

In [24]:
instance = [
  [0, 0, 3, 0, 2, 0, 6, 0, 0],
  [9, 0, 0, 3, 0, 5, 0, 0, 1],
  [0, 0, 1, 8, 0, 6, 4, 0, 0],
  [0, 0, 8, 1, 0, 2, 9, 0, 0],
  [7, 0, 0, 0, 0, 0, 0, 0, 8],
  [0, 0, 6, 7, 0, 8, 2, 0, 0],
  [0, 0, 2, 6, 0, 9, 5, 0, 0],
  [8, 0, 0, 2, 0, 3, 0, 0, 9],
  [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

sudoku = SudokuCSP(instance)
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    inference=forward_checking
    )
sudoku.display(solution)

-------------------------------------
| 4 | 8 | 3 | 9 | 2 | 1 | 6 | 5 | 7 |
-------------------------------------
| 9 | 6 | 7 | 3 | 4 | 5 | 8 | 2 | 1 |
-------------------------------------
| 2 | 5 | 1 | 8 | 7 | 6 | 4 | 9 | 3 |
-------------------------------------
| 5 | 4 | 8 | 1 | 3 | 2 | 9 | 7 | 6 |
-------------------------------------
| 7 | 2 | 9 | 5 | 6 | 4 | 1 | 3 | 8 |
-------------------------------------
| 1 | 3 | 6 | 7 | 9 | 8 | 2 | 4 | 5 |
-------------------------------------
| 3 | 7 | 2 | 6 | 8 | 9 | 5 | 1 | 4 |
-------------------------------------
| 8 | 1 | 4 | 2 | 5 | 3 | 7 | 6 | 9 |
-------------------------------------
| 6 | 9 | 5 | 4 | 1 | 7 | 3 | 8 | 2 |
-------------------------------------


In [25]:
# no heuristics/inference
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku
    )
runtime1 = time.time() - start
sudoku.display(solution)

# with variable selection heuristics
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv
    )
runtime2 = time.time() - start
sudoku.display(solution)

# with value selection heuristics
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    order_domain_values=lcv
    )
runtime3 = time.time() - start
sudoku.display(solution)

# with variable and value selection heuristics
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv
    )
runtime4 = time.time() - start
sudoku.display(solution)

# with inference
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    inference=forward_checking
    )
runtime5 = time.time() - start
sudoku.display(solution)

# with heuristics and inference
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    inference=forward_checking
    )
runtime6 = time.time() - start
sudoku.display(solution)

print('Runtime 1: %.2fs' % (runtime1))
print('Runtime 2: %.2fs' % (runtime2))
print('Runtime 3: %.2fs' % (runtime3))
print('Runtime 4: %.2fs' % (runtime4))
print('Runtime 5: %.2fs' % (runtime5))
print('Runtime 6: %.2fs' % (runtime6))

-------------------------------------
| 4 | 8 | 3 | 9 | 2 | 1 | 6 | 5 | 7 |
-------------------------------------
| 9 | 6 | 7 | 3 | 4 | 5 | 8 | 2 | 1 |
-------------------------------------
| 2 | 5 | 1 | 8 | 7 | 6 | 4 | 9 | 3 |
-------------------------------------
| 5 | 4 | 8 | 1 | 3 | 2 | 9 | 7 | 6 |
-------------------------------------
| 7 | 2 | 9 | 5 | 6 | 4 | 1 | 3 | 8 |
-------------------------------------
| 1 | 3 | 6 | 7 | 9 | 8 | 2 | 4 | 5 |
-------------------------------------
| 3 | 7 | 2 | 6 | 8 | 9 | 5 | 1 | 4 |
-------------------------------------
| 8 | 1 | 4 | 2 | 5 | 3 | 7 | 6 | 9 |
-------------------------------------
| 6 | 9 | 5 | 4 | 1 | 7 | 3 | 8 | 2 |
-------------------------------------
-------------------------------------
| 4 | 8 | 3 | 9 | 2 | 1 | 6 | 5 | 7 |
-------------------------------------
| 9 | 6 | 7 | 3 | 4 | 5 | 8 | 2 | 1 |
-------------------------------------
| 2 | 5 | 1 | 8 | 7 | 6 | 4 | 9 | 3 |
-------------------------------------
| 5 | 4 | 8 

(fim)